## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откроем файл с данными и изучите общую информацию. 

Описание данных
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

In [45]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [46]:
df = pd.read_csv('/datasets/data.csv')
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [47]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [49]:
print(df['gender'].unique())
print(df['education'].unique())
print(df['dob_years'].unique())
print(df['family_status'].unique())
print(df['income_type'].unique())
print(df['children'].unique())

['F' 'M' 'XNA']
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
[ 1  0  3  2 -1  4 20  5]


In [50]:
df.loc[df['days_employed']<0].count()

children            15906
days_employed       15906
dob_years           15906
education           15906
education_id        15906
family_status       15906
family_status_id    15906
gender              15906
income_type         15906
debt                15906
total_income        15906
purpose             15906
dtype: int64

### Вывод

Данные достаточно "грязные". В том числе обнаружилось:
* В гендере есть непонятное значение XNA
* Уровень образования дублируется КАПСОМ
* Клиентами банка являются также новорождённые - dob = 0
* Также есть аномалии в кол-ве детей - children = 20, и - children = -1
* Аномалии в стаже, например 900 лет, при возрасте 53 года
* Ну и в целом в списке 15 906 работников, у которых days_employed < 0
* 2 174 почему то пропустили ответ о трудовом стаже

### Шаг 2. Предобработка данных

### Обработка пропусков

In [51]:
df[df['days_employed'].isna()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


##### Похоже, что отсутствие данных в колонке days_employed не случайно и корелирует с отсутствием данных в колонке total_income. 
##### Судя по пропущенным данным особой сиситемы или какой то зависимости нет. Похоже проблема техническая - не выгрузилась часть данных или связана с человеческим фактором, когда исполнитель по какой то причине не заполнял часть информации.

In [52]:
# уберём капсы в столбце education для корректности группировки
df['education'] = df['education'].str.lower()

# заполним пропущенные значения дохода соответствующими медианными значениями, сгруппированными по типу дохода и уровню образования
for income_type in df['income_type'].unique():
    for education in df['education'].unique():
        median = df[(df['income_type'] == income_type) & (df['education'] == education)]['total_income'].median()
        df.loc[(df['total_income'].isna()) & (df['income_type'] == income_type) & (df['education'] == education),'total_income'] = median
        
# проделаем аналогичную операцию для столбца days_employed, заменив уровень образования на кол-во прожитых лет
# (как более релевантное значение полю стажа)       
# заодно поправим рабочую отрицательность:
def anti_minus(row):
    if row < 0:
        return row*-1
    return row
df['days_employed'] = df['days_employed'].apply(anti_minus)

for income_type in df['income_type'].unique():
    for dob_years in df['dob_years'].unique():
        median = df[(df['income_type'] == income_type) & (df['dob_years'] == dob_years)]['days_employed'].median()
        df.loc[(df['days_employed'].isna()) & (df['income_type'] == income_type) & (df['dob_years'] == dob_years),'days_employed'] = median
  


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21521 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [54]:
# судя по всему, для 4 значений стажа не нашлось соответстыующей возрастной группы,
#с учётом незначительности, считаю возможным избавится от этих строк

df = df.dropna()

In [55]:
df.loc[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Ещё у нас в данных была ошибка пола - XNA. Возможно вместо галочки в нужном поле был проставлен крестик, кторый затем неверно считали. 
Учитывая, что такая запись только одна, а реальный пол не угадать, можно от этой строчки избавиться.

In [56]:
df = df[df['gender'] != 'XNA']
print(df['gender'].unique())

['F' 'M']


done!

### Вывод

Донные обработаны, приведены к нужным типам.

### Замена типа данных

Информация по количеству отработанных дней точна до секунды, а о получаемом доходе до копейки (копейка рубль бережёт). Нам такая точность ни к чему, поэтому можно перевести float в int

In [57]:
df['days_employed'] = df['days_employed'].astype(int)

In [58]:
df['total_income'] = df['total_income'].astype(int)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21520 entries, 0 to 21524
Data columns (total 12 columns):
children            21520 non-null int64
days_employed       21520 non-null int64
dob_years           21520 non-null int64
education           21520 non-null object
education_id        21520 non-null int64
family_status       21520 non-null object
family_status_id    21520 non-null int64
gender              21520 non-null object
income_type         21520 non-null object
debt                21520 non-null int64
total_income        21520 non-null int64
purpose             21520 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Теперь у нас нет не нужных деталей, а тип данных преобразился в int64

### Обработка дубликатов

In [60]:
df.duplicated().sum()

71

In [61]:
# похоже у нас 71 полный дублёр это примерно 0.3% всех данных
df = df.drop_duplicates().reset_index (drop = True)

### Вывод

Мы избавились от полных дублёров. Скорее всего проблема дублирования носит технический характер, на это следует обратить внимание банковских сотрудников

### Лемматизация

In [62]:
# ранее, в первом блоке мы закрузили нужные библиотеки pymystem3 и collections
# напишем небольшую функцию для построчной лемматизации, которая будет каждую лемматизированную строку сохранять в 
# глобальную переменную:
lemmas = []
def lemming(row):
    global lemmas
    lemmas += m.lemmatize(row)
    other_lemmas = m.lemmatize(row)
    return "".join(other_lemmas).strip()

    
# применим функцию    
df['lemmas'] = df['purpose'].apply(lemming)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба


In [63]:
# посчитаем результат
Counter(lemmas)

Counter({'покупка': 5894,
         ' ': 33565,
         'жилье': 4459,
         '\n': 21449,
         'приобретение': 461,
         'автомобиль': 4305,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2323,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6349,
         'коммерческий': 1311,
         'жилой': 1229,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2229,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

### Вывод

Похоже кредиты берут на 4 цели:
* недвижимость (скорее всего жилая и коммерческая);
* автомобиль;
* образование;
* свадьба.

### Категоризация данных

Разобьём доход (total_income) на категории:'высокий доход','средний доход','низкий доход'

In [64]:
df['total_income'].describe().astype(int)


count      21449
mean      165438
std        98271
min        20667
25%       107499
50%       143707
75%       198301
max      2265604
Name: total_income, dtype: int64

разделим уровень дохода на основании квартилей:
* до 107 499 - низкий доход
* от 107 500 до 198 301 - средний доход
* свыше 198 301 - высокий доход

In [65]:
# подготовим соответствующую функцию:
def profit_evaluator(profit):
    if profit < 107499:
        return 'низкий доход'
    if 107500 <= profit <= 198301:
        return 'средний доход'
    return 'высокий доход'

In [66]:
df['income_grade'] = df['total_income'].apply(profit_evaluator)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,income_grade
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,средний доход


In [67]:
df['income_grade'].value_counts()

средний доход    10725
высокий доход     5363
низкий доход      5361
Name: income_grade, dtype: int64

### Вывод

Исходя из нашей градации, больше всего заёмщиков со средим доходом.

### Шаг 3. Проверим гипотезы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Вспомним что у нас есть записи с минус одним ребенком, это надо поправить. Похожее имеет место техническая ошибка, когда в ответ затесался дефис. Воспользуемся функцией anti_minus

In [68]:
df['children'] = df['children'].apply(anti_minus)
print(df['children'].unique())

[ 1  0  3  2  4 20  5]


Оценим сколько многодетных клиентов.

In [69]:
df[df['children']==20].count()

children            76
days_employed       76
dob_years           76
education           76
education_id        76
family_status       76
family_status_id    76
gender              76
income_type         76
debt                76
total_income        76
purpose             76
lemmas              76
income_grade        76
dtype: int64

не много и не мало чтобы явно отнести к ошибке

In [70]:
df.groupby(['children'])['debt'].sum()

children
0     1063
1      445
2      194
3       27
4        4
5        0
20       8
Name: debt, dtype: int64

Выходит самые необязательные безтеные? Посмотрим на долю:

In [71]:
# то же, но с использованием сводной таблицы:
data_pivot = df.pivot_table(index = 'children', aggfunc = {'debt': 'mean'})*100
data_pivot

,debt
children,
0,7.545964
1,9.165808
2,9.458801
3,8.181818
4,9.756098
5,0.000000
20,10.526316


### Вывод

Разброс между детными и бездетными не такой и большой. Чуть лучше если у вас нет детей, чуть хуже если их 20.


P.S. напомнить кредитному отделу проверить данные по 20-дентым

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [72]:
#подготовим функцию для оценки доли невозврата

def debt_ratio (df,column):
    return df.pivot_table(index = column, aggfunc = {'debt': 'mean'})*100

debt_ratio(df, 'family_status')

,debt
family_status,
Не женат / не замужем,9.754361
в разводе,7.112971
вдовец / вдова,6.569343
гражданский брак,9.351651
женат / замужем,7.546405


### Вывод

Больших разбросов вроде бы нет. Чуть лучше давать овдовевшим, чуть хуже холостым

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [73]:
debt_ratio(df, 'income_grade')

,debt
income_grade,
высокий доход,6.955062
низкий доход,7.964932
средний доход,8.773893


### Вывод

Клиенты с низкими доходами похоже более обязательные чем со средним. Самые обязательные - с высоким доходом, что логично.

- *Как разные цели кредита влияют на его возврат в срок?

In [74]:
#используем для сортировки лем ТОП 4 цели:

def lem_sort(row):
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'жилье' in row:
        return 'жилье'    
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'образование' in row:
        return 'образование'
    if 'свадьба' in row:
        return 'свадьба'
    return 'прочее'


df['new_lem'] = df['lemmas'].apply(lem_sort)

In [75]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,income_grade,new_lem
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,высокий доход,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,средний доход,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,средний доход,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,высокий доход,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,средний доход,свадьба


In [76]:
debt_ratio(df, 'new_lem')

,debt
new_lem,
автомобиль,9.361208
жилье,6.907378
недвижимость,7.465743
образование,9.220035
свадьба,8.006888


### Вывод

Похоже ипотека самый безопасный для банка вид кредита, а вот по кредитам на покупку автомобиля лучше внимательнее проверить доходы.

### Шаг 4. Общий вывод

Входные данные были достаточно сырыми, есть очевидная проблема с NaN в доходах и типе деятельности с которой нужно разобраться. Также нужно верифицировать данные по многодетным - 20 детей выглядит очень подозрительным.

В целом нельзя сказать что какой то фактор заметно влияет на просрочки, все выглядят примерно одинаково.


Идеального заёмщика можно описать как: бездетный вдовец с высоким доходом который берёт кредит на покупку жилья.
Плохой заёмщик: многодетный холостяк со средним доходом, который решил купить автомобиль.